In [20]:
import sys

sys.path.append('../src')
sys.path.append('../tests')
#sys.path.insert(0, '/home/ubuntu/kaggle-planet/deforestation/src')

In [22]:
import os
import cv2
import gc
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

import data_helper
import keras_helper

%matplotlib inline
%config InlineBackend.figure_format = 'retina'